[{'id': 'veq1Bl1DW3UWMekZJUsG1Q', 'alias': 'gramercy-tavern-new-york', 'name': 'Gramercy Tavern', 'image_url': 'https://s3-media4.fl.yelpcdn.com/bphoto/yr_BNXJ57Q66IgSppMy1Qw/o.jpg', 'is_closed': False, 'url': 'https://www.yelp.com/biz/gramercy-tavern-new-york?adjust_creative=CLQ6g_r4luauXIIe1L9OPQ&utm_campaign=yelp_api_v3&utm_medium=api_v3_business_search&utm_source=CLQ6g_r4luauXIIe1L9OPQ', 'review_count': 2500, 'categories': [{'alias': 'newamerican', 'title': 'American (New)'}], 'rating': 4.5, 'coordinates': {'latitude': 40.738527, 'longitude': -73.988417}, 'transactions': [], 'price': '$$$$', 'location': {'address1': '42 E 20th St', 'address2': '', 'address3': '', 'city': 'New York', 'zip_code': '10003', 'country': 'US', 'state': 'NY', 'display_address': ['42 E 20th St', 'New York, NY 10003']}, 'phone': '+12124770777', 'display_phone': '(212) 477-0777', 'distance': 3711.7439058285554}, {'id': '4yPqqJDJOQX69gC66YUDkA', 'alias': 'peter-luger-brooklyn-2', 'name': 'Peter Luger', 'image_

In [62]:
#import re
#tmp = pd.series(['Name'])#
#df[df.NAME.str.contains('Boston', flags=re.IGNORECASE, regex=True)]



,OBJECTID,URL,NAME,the_geom,LINE


In [11]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import operator

df = pd.read_csv('http://web.mta.info/developers/data/nyct/turnstile/turnstile_180630.txt')

df.columns = df.columns.str.strip()

df['real_entries'] = df['ENTRIES'].diff()
df['real_exits'] = df['EXITS'].diff()

df.loc[df['real_entries'] < 0] = np.nan
df.loc[df['real_exits'] < 0] = np.nan
df.loc[df['real_entries'] > 100000] = np.nan
df.loc[df['real_exits'] > 100000] = np.nan

#exitting or postivie flux??
#print(df.columns)

station_1_av = df[df['STATION'] == '1 AV']
#print(df.sort_values(by='real_entries', ascending=False).head(100))

#print(df.groupby(['STATION'])['real_entries', 'real_exits'].mean())

new_df = df.groupby(['STATION'])['real_entries', 'real_exits'].mean()

totals = {}
for index, row in new_df.iterrows():
    totals[index] = row['real_entries'] + row['real_exits']

sorted_totals = sorted(totals.items(), key=operator.itemgetter(1), reverse=True)
print(sorted_totals)

[('ASTORIA DITMARS', 873.8658536585367), ('YORK ST', 838.9833333333333), ('14 ST-UNION SQ', 795.1721258831085), ('NEWARK C', 746.2635327635328), ('MYRTLE AV', 722.6375), ('1 AV', 693.9658536585366), ('W 4 ST-WASH SQ', 687.4715025906736), ('BEDFORD AV', 677.2871794871794), ('GRD CNTRL-42 ST', 652.4405487804878), ('34 ST-HERALD SQ', 641.1924001924002), ('FLUSHING-MAIN', 629.7427797833934), ('8 AV', 629.1356707317074), ('57 ST-7 AV', 626.630081300813), ('TIMES SQ-42 ST', 606.7898406374502), ('PRINCE ST', 605.9610705596107), ('EUCLID AV', 604.0395136778116), ('NEWARK BM BW', 599.2826704545455), ('191 ST', 598.8414634146341), ('NEWKIRK PLAZA', 596.609756097561), ('42 ST-BRYANT PK', 593.1277584204413), ('HOYT-SCHER', 592.3024390243902), ('JOURNAL SQUARE', 591.4009378663541), ('JKSN HT-ROOSVLT', 590.2830188679245), ('GRAND ST', 577.6697936210131), ('PARKSIDE AV', 576.1056910569106), ('72 ST', 575.6725067385445), ('103 ST-CORONA', 569.6138888888889), ('NOSTRAND AV', 563.8243902439025), ('53 ST

In [18]:
locations = []
for key, value in sorted_totals:
    locations.append(key)
print(locations[0])

ASTORIA DITMARS


In [31]:
from yelpapi import YelpAPI
import io, json
# read API keys (note please make your own config_secret.json)
#{
# "api_key":"YOUR API KEY HERE"
#}
with io.open('config_secret.json') as cred:
    creds = json.load(cred)

# input location     
yelp_api = YelpAPI(**creds)
for loc in locations: #example, can you use many 
    
    args = {
    'location': loc,
    'limit': 50,
    'categories': 'coffee,restaurants',
    #'latitude': loc[0],
    #'longitude': loc[1],
    'radius_filter': '241.40160000000003',
    'price': '4'
    }
    search_results = yelp_api.search_query(**args)
    break
print(search_results.items())





dict_items([('businesses', [{'id': 'wSGZA3DOFSUvALZRKCM_Sw', 'alias': 'tanoshi-sushi-sake-bar-new-york', 'name': 'Tanoshi Sushi Sake Bar', 'image_url': 'https://s3-media3.fl.yelpcdn.com/bphoto/9Lq02-rtpaTJh_gnGfoSJw/o.jpg', 'is_closed': False, 'url': 'https://www.yelp.com/biz/tanoshi-sushi-sake-bar-new-york?adjust_creative=CLQ6g_r4luauXIIe1L9OPQ&utm_campaign=yelp_api_v3&utm_medium=api_v3_business_search&utm_source=CLQ6g_r4luauXIIe1L9OPQ', 'review_count': 515, 'categories': [{'alias': 'sushi', 'title': 'Sushi Bars'}, {'alias': 'japanese', 'title': 'Japanese'}, {'alias': 'seafood', 'title': 'Seafood'}], 'rating': 4.5, 'coordinates': {'latitude': 40.767568, 'longitude': -73.952815}, 'transactions': ['restaurant_reservation'], 'price': '$$$$', 'location': {'address1': '1372 York Ave', 'address2': None, 'address3': '', 'city': 'New York', 'zip_code': '10021', 'country': 'US', 'state': 'NY', 'display_address': ['1372 York Ave', 'New York, NY 10021']}, 'phone': '+19172658254', 'display_phone'

In [ ]:
#import pandas as pd
#import numpy as np
#import re
#import csv
#locations = []

#csv_input = 'DOITT_SUBWAY_ENTRANCE_01_13SEPT2010.csv'
#df = pd.read_csv(csv_input)
#for loc in df.the_geom:
#    regex = re.compile('\d+.\d+')
#    locations.append(regex.findall(loc))   